# Optuna lightGBM

In [1]:
import pandas as pd
import numpy as np

import lightgbm as lgb # conda install -c conda-forge lightgbm
import sklearn.metrics
from sklearn.model_selection import train_test_split

import optuna

Duplicate key in file WindowsPath('C:/Users/iiats/Anaconda3/envs/Python37-gpu/lib/site-packages/matplotlib/mpl-data/matplotlibrc'), line 250 ('font.family:  IPAexGothic')


In [2]:
df = pd.read_csv("./data/mobile_train.csv")
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [3]:
X = df.drop("price_range", axis=1).values
y = df.price_range.values

In [4]:
X

array([[8.420e+02, 0.000e+00, 2.200e+00, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       [1.021e+03, 1.000e+00, 5.000e-01, ..., 1.000e+00, 1.000e+00,
        0.000e+00],
       [5.630e+02, 1.000e+00, 5.000e-01, ..., 1.000e+00, 1.000e+00,
        0.000e+00],
       ...,
       [1.911e+03, 0.000e+00, 9.000e-01, ..., 1.000e+00, 1.000e+00,
        0.000e+00],
       [1.512e+03, 0.000e+00, 9.000e-01, ..., 1.000e+00, 1.000e+00,
        1.000e+00],
       [5.100e+02, 1.000e+00, 2.000e+00, ..., 1.000e+00, 1.000e+00,
        1.000e+00]])

In [5]:
y

array([1, 2, 2, ..., 3, 0, 3], dtype=int64)

In [55]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25)
dtrain = lgb.Dataset(X_train, label=y_train)

## lightGBM default

In [7]:
# default parameters
param = {
#         'objective': 'multiclass',
#         'metric': 'multi_logloss',
#         'num_class': 4,
        "verbosity": -1,
        "boosting_type": "gbdt"
}

In [8]:
%%time
gbm = lgb.train(param, dtrain)
preds = gbm.predict(X_val)
pred_labels = np.rint(preds)
accuracy = sklearn.metrics.accuracy_score(y_val, pred_labels)
print(accuracy)

0.89
Wall time: 78 ms


## Optuna

In [9]:
# FYI: Objective functions can take additional arguments
# (https://optuna.readthedocs.io/en/stable/faq.html#objective-func-additional-args).

def objective(trial):
    train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.25)
    dtrain = lgb.Dataset(train_x, label=train_y)

    param = {
#         'objective': 'multiclass',
#         'metric': 'multi_logloss',
#         'num_class': 4,
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }

    gbm = lgb.train(param, dtrain)
    preds = gbm.predict(valid_x)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(valid_y, pred_labels)
    return accuracy

In [10]:
%%time
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

[I 2020-12-12 00:53:29,914] A new study created in memory with name: no-name-97d9b139-2963-4aed-af32-0c317d0c6a18
[I 2020-12-12 00:53:29,963] Trial 0 finished with value: 0.922 and parameters: {'lambda_l1': 1.8038813329097914e-06, 'lambda_l2': 0.0007339467686536127, 'num_leaves': 170, 'feature_fraction': 0.6677230920861777, 'bagging_fraction': 0.8504478310486803, 'bagging_freq': 2, 'min_child_samples': 99}. Best is trial 0 with value: 0.922.
[I 2020-12-12 00:53:30,050] Trial 1 finished with value: 0.898 and parameters: {'lambda_l1': 3.813161477198156e-08, 'lambda_l2': 2.7613638635330355e-06, 'num_leaves': 222, 'feature_fraction': 0.4025933939495965, 'bagging_fraction': 0.9934642063598293, 'bagging_freq': 5, 'min_child_samples': 31}. Best is trial 0 with value: 0.922.
[I 2020-12-12 00:53:30,090] Trial 2 finished with value: 0.908 and parameters: {'lambda_l1': 0.2668676363515926, 'lambda_l2': 3.44286367737828e-08, 'num_leaves': 68, 'feature_fraction': 0.6873706018761923, 'bagging_fractio

[I 2020-12-12 00:53:31,609] Trial 24 finished with value: 0.904 and parameters: {'lambda_l1': 0.008746511154040567, 'lambda_l2': 0.00019199354559650556, 'num_leaves': 253, 'feature_fraction': 0.6323548419575504, 'bagging_fraction': 0.6011396300202396, 'bagging_freq': 4, 'min_child_samples': 61}. Best is trial 14 with value: 0.93.
[I 2020-12-12 00:53:31,677] Trial 25 finished with value: 0.902 and parameters: {'lambda_l1': 0.00018755493767445573, 'lambda_l2': 8.112313925039527e-06, 'num_leaves': 234, 'feature_fraction': 0.44413708780395855, 'bagging_fraction': 0.9078679012603614, 'bagging_freq': 4, 'min_child_samples': 76}. Best is trial 14 with value: 0.93.
[I 2020-12-12 00:53:31,744] Trial 26 finished with value: 0.896 and parameters: {'lambda_l1': 0.001478154457545992, 'lambda_l2': 2.665530390923167e-07, 'num_leaves': 183, 'feature_fraction': 0.5658491436594366, 'bagging_fraction': 0.6991734331441667, 'bagging_freq': 5, 'min_child_samples': 63}. Best is trial 14 with value: 0.93.
[I 

[I 2020-12-12 00:53:33,120] Trial 48 finished with value: 0.944 and parameters: {'lambda_l1': 6.65045749742121e-08, 'lambda_l2': 1.7450120009531438, 'num_leaves': 80, 'feature_fraction': 0.5404677367957086, 'bagging_fraction': 0.9558218748096161, 'bagging_freq': 7, 'min_child_samples': 85}. Best is trial 35 with value: 0.948.
[I 2020-12-12 00:53:33,200] Trial 49 finished with value: 0.866 and parameters: {'lambda_l1': 9.960486657268373e-08, 'lambda_l2': 1.6886510291732424, 'num_leaves': 77, 'feature_fraction': 0.4263406674644162, 'bagging_fraction': 0.9558246774332987, 'bagging_freq': 7, 'min_child_samples': 42}. Best is trial 35 with value: 0.948.
[I 2020-12-12 00:53:33,268] Trial 50 finished with value: 0.908 and parameters: {'lambda_l1': 1.6700942880015027e-08, 'lambda_l2': 8.905036347905309, 'num_leaves': 48, 'feature_fraction': 0.5359307276273417, 'bagging_fraction': 0.40746328562915357, 'bagging_freq': 6, 'min_child_samples': 22}. Best is trial 35 with value: 0.948.
[I 2020-12-12

[I 2020-12-12 00:53:34,650] Trial 72 finished with value: 0.898 and parameters: {'lambda_l1': 8.103646051336233e-06, 'lambda_l2': 0.010624789671752281, 'num_leaves': 86, 'feature_fraction': 0.4784916227023495, 'bagging_fraction': 0.9369709173087093, 'bagging_freq': 6, 'min_child_samples': 90}. Best is trial 35 with value: 0.948.
[I 2020-12-12 00:53:34,712] Trial 73 finished with value: 0.902 and parameters: {'lambda_l1': 0.00039368400371165934, 'lambda_l2': 1.0974812655072892e-08, 'num_leaves': 132, 'feature_fraction': 0.43981488980468375, 'bagging_fraction': 0.8097001207667083, 'bagging_freq': 4, 'min_child_samples': 79}. Best is trial 35 with value: 0.948.
[I 2020-12-12 00:53:34,779] Trial 74 finished with value: 0.914 and parameters: {'lambda_l1': 3.826543956270656e-06, 'lambda_l2': 0.001195339637269723, 'num_leaves': 101, 'feature_fraction': 0.4995444826037188, 'bagging_fraction': 0.9814895638450841, 'bagging_freq': 6, 'min_child_samples': 70}. Best is trial 35 with value: 0.948.
[

[I 2020-12-12 00:53:36,191] Trial 96 finished with value: 0.912 and parameters: {'lambda_l1': 3.697026252670435e-05, 'lambda_l2': 4.415511436567799e-08, 'num_leaves': 127, 'feature_fraction': 0.5058054030212299, 'bagging_fraction': 0.8706919264630797, 'bagging_freq': 5, 'min_child_samples': 73}. Best is trial 35 with value: 0.948.
[I 2020-12-12 00:53:36,255] Trial 97 finished with value: 0.93 and parameters: {'lambda_l1': 0.00012461151686913284, 'lambda_l2': 0.010297547860655643, 'num_leaves': 111, 'feature_fraction': 0.5692397526112427, 'bagging_fraction': 0.9881439922665115, 'bagging_freq': 7, 'min_child_samples': 98}. Best is trial 35 with value: 0.948.
[I 2020-12-12 00:53:36,323] Trial 98 finished with value: 0.916 and parameters: {'lambda_l1': 5.331372167912013e-06, 'lambda_l2': 9.478179495352712e-05, 'num_leaves': 66, 'feature_fraction': 0.463917661696976, 'bagging_fraction': 0.9992077445241618, 'bagging_freq': 6, 'min_child_samples': 76}. Best is trial 35 with value: 0.948.
[I 2

Wall time: 6.48 s


In [11]:
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials: 100
Best trial:
  Value: 0.948
  Params: 
    lambda_l1: 0.00011587891848531416
    lambda_l2: 9.678478038501493e-08
    num_leaves: 91
    feature_fraction: 0.4906156854247304
    bagging_fraction: 0.9980589718437096
    bagging_freq: 5
    min_child_samples: 96


In [12]:
trial.params

{'lambda_l1': 0.00011587891848531416,
 'lambda_l2': 9.678478038501493e-08,
 'num_leaves': 91,
 'feature_fraction': 0.4906156854247304,
 'bagging_fraction': 0.9980589718437096,
 'bagging_freq': 5,
 'min_child_samples': 96}

In [13]:
%%time
gbm = lgb.train(trial.params, dtrain)
preds = gbm.predict(X_val)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000696 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1361
[LightGBM] [Info] Number of data points in the train set: 1500, number of used features: 20
[LightGBM] [Info] Start training from score 1.493333
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

In [14]:
pred_labels = np.rint(preds)
pred_labels

array([ 3.,  0.,  3.,  3.,  0.,  2.,  3.,  3.,  3.,  2.,  3.,  1., -0.,
        2.,  3.,  2.,  3.,  3.,  0.,  0.,  1.,  1.,  3.,  2.,  2.,  2.,
        1.,  0.,  0.,  3.,  1.,  0.,  1.,  0.,  3.,  3.,  2.,  2.,  2.,
        2.,  3.,  0.,  2.,  3.,  0.,  3.,  1.,  1.,  1., -0.,  1.,  2.,
        1.,  3.,  1.,  1.,  2.,  1.,  1.,  0.,  3.,  1.,  2.,  1.,  3.,
        3.,  1.,  0.,  3.,  2.,  0.,  2.,  0.,  0.,  0.,  3.,  2.,  2.,
       -0.,  1.,  1.,  0., -0.,  2.,  1.,  2.,  3.,  3.,  1.,  2.,  0.,
        1.,  2.,  1.,  0.,  0.,  3.,  3.,  2.,  3.,  0.,  2.,  1.,  3.,
       -0.,  3.,  3.,  2.,  2.,  2.,  0.,  2., -0.,  1.,  2.,  2., -0.,
        3.,  1.,  3.,  0.,  1.,  1.,  2.,  3.,  2.,  0.,  1.,  1., -0.,
        1.,  3.,  3.,  3.,  3.,  3.,  0.,  3.,  0.,  3.,  2.,  2.,  2.,
        3.,  1.,  1.,  3.,  3.,  2.,  0.,  2.,  2.,  1.,  3.,  0.,  2.,
        2.,  3.,  2.,  1.,  1.,  3.,  3.,  2., -0.,  2.,  2.,  1.,  2.,
        3.,  0.,  1.,  1.,  1.,  0.,  1.,  3.,  2.,  3.,  0.,  0

In [15]:
accuracy = sklearn.metrics.accuracy_score(y_val, pred_labels)
accuracy

0.916

## Optuna lightGBM Tuner

In [16]:
import optuna.integration.lightgbm as lgb_o

In [17]:
# LightGBM用のデータセットに変換
train = lgb_o.Dataset(X_train, y_train)
val = lgb_o.Dataset(X_val, y_val)

In [18]:
%%time
# ハイパーパラメータサーチ&モデル構築
params = {'objective': 'multiclass',
          'metric': 'multi_logloss',
          'num_class': 4,
          'random_seed':0} 

gbm_o = lgb_o.train(params,
                    train,
                    valid_sets=val,
                    early_stopping_rounds=200,
                    verbose_eval=10000,)

[I 2020-12-12 00:53:36,578] A new study created in memory with name: no-name-6e56ac85-542c-4a7b-9ffe-0cca44017b54
feature_fraction, val_score: inf:   0%|                                                          | 0/7 [00:00<?, ?it/s]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000348 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1361
[LightGBM] [Info] Number of data points in the train set: 1500, number of used features: 20
[LightGBM] [Info] Start training from score -1.394327
[LightGBM] [Info] Start training from score -1.365185
[LightGBM] [Info] Start training from score -1.378326
[LightGBM] [Info] Start training from score -1.407859
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 200 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

feature_fraction, val_score: 0.227263:  14%|######4                                      | 1/7 [00:00<00:04,  1.21it/s]

No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with po

...

Early stopping, best iteration is:
[332]	valid_0's multi_logloss: 0.175501
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000419 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1361
[LightGBM] [Info] Number of data points in the train set: 1500, number of used features: 20
[LightGBM] [Info] Start training from score -1.394327
[LightGBM] [Info] Start training from score -1.365185
[LightGBM] [Info] Start training from score -1.378326
[LightGBM] [Info] Start training from score -1.407859
Training until validation scores don't improve for 200 rounds


min_data_in_leaf, val_score: 0.168989:  80%|####################################         | 4/5 [00:02<00:00,  1.97it/s]

Early stopping, best iteration is:
[402]	valid_0's multi_logloss: 0.17678
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000413 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1361
[LightGBM] [Info] Number of data points in the train set: 1500, number of used features: 20
[LightGBM] [Info] Start training from score -1.394327
[LightGBM] [Info] Start training from score -1.365185
[LightGBM] [Info] Start training from score -1.378326
[LightGBM] [Info] Start training from score -1.407859
Training until validation scores don't improve for 200 rounds


min_data_in_leaf, val_score: 0.168989: 100%|#############################################| 5/5 [00:02<00:00,  1.97it/s]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Early stopping, best iteration is:
[352]	valid_0's multi_logloss: 0.185203
Wall time: 43.2 s


In [19]:
# ベストパラメータの取得
best_params = gbm_o.params
print("  Params: ")
for key, value in best_params.items():
    print("    {}: {}".format(key, value))

  Params: 
    objective: multiclass
    metric: multi_logloss
    num_class: 4
    random_seed: 0
    feature_pre_filter: False
    lambda_l1: 1.181404696897328e-05
    lambda_l2: 2.405082416410275e-07
    num_leaves: 5
    feature_fraction: 1.0
    bagging_fraction: 0.7244983918387866
    bagging_freq: 7
    min_child_samples: 20
    num_iterations: 1000
    early_stopping_round: 200


In [20]:
# 調整後モデルで予測の実行
y_trainval_pred = gbm_o.predict(X_train,num_iteration=gbm_o.best_iteration)
preds = gbm_o.predict(X_val,num_iteration=gbm_o.best_iteration)

In [21]:
preds

array([[3.93540232e-14, 8.71005126e-14, 1.38551666e-05, 9.99986145e-01],
       [9.99977064e-01, 2.29361204e-05, 4.54619944e-13, 1.14555895e-14],
       [6.36123888e-11, 4.08262107e-10, 1.13597243e-04, 9.99886402e-01],
       ...,
       [6.02503941e-05, 7.83805860e-06, 8.35662327e-01, 1.64269585e-01],
       [1.07716503e-06, 9.99729641e-01, 2.69281402e-04, 1.57663414e-11],
       [1.70371297e-08, 2.22066688e-08, 5.19296862e-04, 9.99480664e-01]])

In [22]:
len(preds)

500

In [23]:
# 実験
preds[100]

array([3.87965614e-01, 6.12034296e-01, 8.79213818e-08, 2.12571271e-09])

In [24]:
lst = preds[100].tolist()
lst

[0.3879656139419829,
 0.6120342960109226,
 8.792138177678022e-08,
 2.1257127092391435e-09]

In [25]:
lst.index(max(lst))

1

In [26]:
# リストの最大値のインデックスを取得する方法
# https://hibiki-press.tech/python/max_min_index/586
# NumPy配列ndarrayをリストに変換: tolist()
# https://note.nkmk.me/python-numpy-list/

pred_labels = []

for i in range(len(preds)):
    lst = preds[i].tolist()
    index = lst.index(max(lst))
    pred_labels.append(index)

In [27]:
pred_labels = np.array(pred_labels)
pred_labels

array([3, 0, 3, 3, 1, 2, 3, 3, 3, 2, 3, 1, 0, 2, 2, 2, 3, 3, 0, 0, 1, 1,
       3, 2, 2, 2, 1, 0, 0, 3, 1, 0, 2, 0, 3, 3, 2, 2, 2, 2, 3, 1, 2, 2,
       0, 3, 1, 1, 1, 0, 1, 2, 1, 3, 1, 1, 1, 1, 1, 0, 3, 1, 2, 2, 3, 3,
       1, 0, 3, 2, 0, 2, 0, 0, 0, 3, 3, 2, 0, 1, 1, 0, 0, 2, 1, 2, 3, 3,
       1, 2, 1, 1, 2, 1, 0, 0, 3, 3, 2, 3, 1, 2, 1, 3, 0, 3, 3, 2, 2, 3,
       0, 2, 0, 1, 3, 2, 0, 3, 1, 3, 0, 2, 1, 2, 3, 1, 0, 1, 1, 0, 1, 2,
       3, 3, 3, 3, 0, 3, 0, 3, 2, 2, 2, 3, 1, 0, 3, 3, 2, 0, 2, 2, 0, 3,
       0, 3, 2, 2, 2, 1, 1, 3, 3, 1, 0, 2, 2, 1, 2, 3, 0, 1, 1, 1, 0, 1,
       3, 2, 3, 0, 0, 3, 0, 3, 1, 3, 3, 3, 3, 1, 0, 1, 3, 0, 2, 1, 2, 1,
       1, 0, 3, 2, 0, 0, 3, 2, 0, 3, 0, 1, 2, 0, 1, 2, 0, 3, 1, 3, 2, 3,
       3, 3, 1, 2, 1, 0, 1, 1, 2, 0, 3, 0, 3, 1, 3, 2, 0, 3, 2, 2, 2, 2,
       3, 1, 2, 3, 3, 1, 1, 1, 2, 3, 1, 2, 0, 0, 3, 0, 3, 0, 1, 3, 0, 1,
       3, 0, 2, 2, 1, 0, 1, 0, 1, 0, 0, 2, 2, 1, 3, 1, 0, 2, 1, 2, 2, 2,
       2, 0, 0, 2, 3, 2, 3, 1, 2, 1, 1, 1, 3, 3, 1,

In [28]:
len(pred_labels)

500

In [29]:
y_val

array([3, 0, 3, 3, 1, 2, 3, 3, 3, 3, 3, 1, 0, 2, 3, 2, 3, 3, 0, 0, 1, 1,
       3, 2, 2, 2, 1, 0, 0, 3, 1, 0, 1, 0, 3, 3, 2, 2, 2, 1, 3, 0, 2, 2,
       0, 2, 1, 1, 1, 0, 1, 2, 1, 3, 1, 1, 1, 1, 1, 0, 3, 1, 2, 2, 3, 3,
       1, 0, 3, 2, 0, 2, 1, 0, 0, 3, 2, 2, 0, 1, 1, 0, 0, 2, 1, 2, 3, 3,
       1, 2, 1, 1, 2, 1, 0, 0, 3, 3, 2, 3, 0, 1, 1, 3, 0, 3, 3, 2, 2, 3,
       0, 2, 0, 1, 3, 2, 0, 3, 1, 3, 0, 1, 1, 2, 3, 1, 0, 1, 1, 0, 1, 3,
       3, 3, 3, 3, 0, 3, 0, 3, 2, 3, 2, 3, 1, 0, 3, 3, 2, 0, 2, 2, 1, 3,
       0, 3, 2, 2, 2, 1, 1, 3, 3, 2, 0, 2, 2, 1, 2, 3, 0, 2, 1, 1, 0, 1,
       3, 2, 3, 0, 0, 3, 0, 3, 1, 3, 2, 3, 3, 1, 0, 1, 3, 0, 2, 1, 2, 1,
       1, 0, 3, 2, 0, 0, 3, 2, 0, 3, 0, 1, 2, 0, 1, 2, 0, 3, 1, 3, 2, 2,
       3, 3, 1, 2, 1, 0, 1, 2, 2, 0, 3, 0, 3, 1, 3, 2, 0, 3, 2, 1, 2, 2,
       3, 1, 2, 3, 3, 1, 1, 2, 2, 3, 0, 2, 0, 0, 3, 0, 3, 0, 1, 3, 0, 1,
       3, 0, 2, 2, 2, 0, 1, 0, 1, 0, 0, 2, 2, 1, 2, 1, 0, 2, 1, 2, 2, 2,
       2, 0, 0, 2, 3, 2, 3, 1, 2, 1, 1, 1, 3, 3, 1,

In [30]:
len(y_val)

500

In [31]:
accuracy = sklearn.metrics.accuracy_score(y_val, pred_labels)
print(accuracy)

0.932


## Summary

### Random Forest
Deffault (cv=5)：0.8745 (1.38s)  
Grid search:   0.8885 (2min 50s)  
Random search: 0.8895 (1min 53s)  
Optuna:        0.8845 (1min 24s)  
Optuna(2): 0.880 (1min 12s)  
Bayesian optimization with gaussian process: KFCV: 0.905, StratifiedCV: 0.9065 (4min 15s)  
hyperopt: KFCV: 0.898, StratifiedCV: 0.898 (2min 51s) 

### lightGBM  
#### Hold out 法 (25%)
Deffault: 0.89 (78ms)  
Optuna: 0.916 (6.48s)  
Optuna lightGBM Tuner: 0.932 (43.2s)